In [1]:
from pattern.web import Wikipedia, plaintext
from nltk.util import ngrams
from string import punctuation
from collections import Counter, defaultdict
import unittest
from numpy import log
import codecs
import re

ImportError: No module named 'pattern'

In [ ]:
def clean_text(s):
    intro_punc = '"#$%&\'()*+,-/:;<=>@[\\]^_`{|}~'
    s = re.sub('['+re.escape(intro_punc)+']+(?:\s|$)',' ',s)
    s = re.sub('(?:\s|^)['+re.escape(intro_punc)+']+',' ',s)
    s = re.sub('\s+',' ',s)
    s = s.lower().strip()
    return s

In [ ]:
class WikiParser:
    def __init__(self):
        pass
  
    def get_articles(self, start, depth, max_count):
        article = Wikipedia().article(start)
        links = article.links
        list_of_strings = []
        for link in links:
            s = Wikipedia().article(link)
            if s is not None:
                s = clean_text(plaintext(s.source))
                list_of_strings.append(s)
        return list_of_strings

In [ ]:
class TextStatistics:
    def __init__(self, articles):
        self.list = articles
        pass
    
    def get_top_3grams(self, n, use_idf=False):
        dict_trigrams = {}
        new_list = []
        sentences = []
        
        for article in self.list:
            new = ''.join([i for i in article if (not i.isdigit())])
            sentences += re.split('[\.?!]', new)

        for sentence in sentences:
            sentence_trigrams = list(trigrams(sentence))
            for sentence_trigram in sentence_trigrams:
                sentence_trigram_join = ''.join(sentence_trigram)
                if sentence_trigram_join in dict_trigrams:
                    dict_trigrams[sentence_trigram_join] += 1
                else:
                    dict_trigrams[sentence_trigram_join] = 1
                               
        if use_idf == True:
            for gram in dict_trigrams:
                gram_in_sentences = [gram in i for i in sentences].count(True)
                idf = math.log(len(sentences) / float(gram_in_sentences))
                dict_trigrams[gram] *= idf
        items_sorted = sorted(dict_trigrams.items(), key=lambda pair:(pair[1], pair[0]), reverse=True)
        items_sorted_trigrs = [i[0] for i in items_sorted]
        items_sorted_freqs = [i[1] for i in items_sorted]
        list_of_3grams_in_descending_order_by_freq = items_sorted_trigrs[:n]
        list_of_their_corresponding_freq = items_sorted_freqs[:n]
        return (list_of_3grams_in_descending_order_by_freq, list_of_their_corresponding_freq)
    
    def get_top_words(self, n, use_idf=False):
        dict_words = {}
        pos_articles = set(['a', 'an', 'the'])
        some_stop_words = ['and', 'are', 'that', 'to', 'this', 'which']
        pos_prepositions = set(['aboard', 'about', 'above', 'across', 'afore', 'after', 'against', 'along', 'amid', 'amidst', 'among', 'amongst', 'around', 'as', 'aside', 'aslant', 'astride', 'at', 'athwart', 'atop', 'between', 'before', 'behind', 'below', 'beneath', 'beside', 'besides', 'between', 'betwixt', 'beyond', 'by', 'circa', 'despite','down', 'except', 'for', 'from', 'in', 'inside', 'into', 'like', 'near', 'neath', 'next', 'of', 'off', 'on', 'opposite', 'out', 'outside', 'over', 'per', 'through', 'till'', toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'with', 'without'])
        sentences = []
        
        for article in self.list:
            new = ''.join([i for i in article if not i.isdigit()])
            sentences += re.split('[\.?!]', new)
            
        for sentence in sentences:
            sentence_words = [word for word in sentence.split() if ((not (word in pos_articles)) and (not (word in pos_prepositions)))]

            for sentence_word in sentence_words:
                if sentence_word in dict_words:
                    dict_words[sentence_word] += 1
                else:
                    dict_words[sentence_word] = 1
                    
        if use_idf == True:
            for word in dict_words:
                word_in_sentences = [word in i for i in sentences].count(True)
                idf = math.log(len(sentences) / float(word_in_sentences))
                dict_words[word] *= idf
        items_sorted = sorted(dict_words.items(), key=lambda pair:(pair[1], pair[0]), reverse=True)
        items_sorted_words = [i[0] for i in items_sorted]
        items_sorted_freqs = [i[1] for i in items_sorted]
        list_of_words_in_descending_order_by_freq = items_sorted_words[:n]
        list_of_their_corresponding_freq = items_sorted_freqs[:n]
        return (list_of_words_in_descending_order_by_freq, list_of_their_corresponding_freq)

In [ ]:
class Experiment:
    def __init__(self):
        pass

    def show_results(self):
        parser = WikiParser()
        articles = parser.get_articles('Natural language processing',1,1)
        stats = TextStatistics(articles)
        a_stats = (stats.get_top_3grams(20,use_idf=True),
                   stats.get_top_words(20,use_idf=True))
        res = 'top-20 n-grams for articles from NLP article:\n%s\n\ntop-20 words for articles from NLP article:\n%s'
        print(res % ('\n'.join([x+' - '+str(y) for x,y in zip(*a_stats[0])]),
                     '\n'.join([x+' - '+str(y) for x,y in zip(*a_stats[1])])))

In [ ]:
try_exp = Experiment()
try_exp.show_results()